In [19]:
def keyLength(ciph_loi):
    """
    Receive the ciphertext as a list of integers (for ease of use).
    Computes the estimated key length based on the sum of char. frequencies.
    Returns the estimated key length.
    """
    sum_qi_list = []
    # Loop over possible key lengths. 
    for n in range(1, 14):
        # Get every nth char from list.
        ciph_loi_nth = ciph_loi[::n]
        # Construct ascii frequency list of nth characters.
        freq_list = [0] * 256
        for m in ciph_loi_nth:
            freq_list[m] += 1 / len(ciph_loi_nth)
        # Find the sum of qi^2 for each i, the highest value is most 
        # likely the key length.
        sum_qi = 0
        for k in freq_list:
            sum_qi += pow(k, 2)
        sum_qi_list.append(sum_qi)

    return sum_qi_list.index(max(sum_qi_list)) + 1


def findCandidatesMatch(ith_byte_list, freq_lc_english):
    sum_qipi_list = []
    # try every possible plaintext candidate.
    for m in range(0, 256):
        freq_list = [0] * 256

        # decrypt using m XOR i (m being the value of n-th bit of the key)
        candidate = [m ^ i for i in ith_byte_list.copy()]

        # rule out invalid candidates
        if(any((i < 32 or i > 127 or 
                (i >= 48 and i <= 57)) for i in candidate)):
            continue

        # find the frequency of all characters for the candidate.
        for i in candidate:
            freq_list[i] += 1

        # Assume the candidate is incorrect if no letters appear.
        if(sum(freq_list[97:123]) == 0):
            continue
        
        # update frequency list to only include 
        # frequency of lower case chars.
        lowercase_freq = sum(freq_list[97:123])
        freq_list_lowercase = [i / lowercase_freq
                                for i in freq_list[97:123]]

        # Find the sum of qi * pi, should be around 0.065.
        sum_qipi = 0
        for i in range(0, 26):
            sum_qipi += freq_list_lowercase[i] * freq_lc_english[i]

        sum_qipi_list.append((sum_qipi, m))
    return sum_qipi_list


def findKey(key_len, ciph_loi):
    """
    Receives the estimated key length and ciphertext
    Computes the most likely key by finding the candidate message whose 
    lowercase character frequency best match the english alphabet frequency.
    Returns the estimated key.
    """
    # English letter frequency from:
    # https://www.coursera.org/learn/cryptography/lecture/01t8O/breaking-the-vigenere-cipher
    freq_lc_english = [0.082, 0.015, 0.028, 0.043, 0.127, 0.022, 0.020, 0.061,
                       0.070, 0.002, 0.008, 0.040, 0.024, 0.067, 0.075, 0.019,
                       0.001, 0.060, 0.063, 0.091, 0.028, 0.010, 0.024, 0.002,
                       0.020, 0.001]
    key = []
    # look at each position of the key
    for n in range(0, key_len):
        # get the ith bytes for a key position
        ith_byte_list = ciph_loi[n::key_len]
        sum_qipi_list = findCandidatesMatch(ith_byte_list, freq_lc_english)
        
        # Based on: https://stackoverflow.com/questions/12141150/
        key.append(min(sum_qipi_list, key=lambda x:abs(x[0] - 0.065))[1])
    return key


def decrypt(key, ciph_loi):
    """
    Receives the key and ciphertext
    Decrypts the ciphertext by XOR'ing the key, print the message.
    """
    ciph_solved = []
    
    for i in range(0, len(ciph_loi)):
        ciph_solved.append(chr(key[i%len(key)] ^ ciph_loi[i]))
    print(''.join(ciph_solved))


def main():
    """
    From a given file, estimate the key length and the key.
    Decrypt and print the original message using the key.
    """
    with open('ciphertext.txt', 'r') as ciphtext:
        ciph = ciphtext.read().rstrip('\n');
        # convert chiptertext string to list of integers (for ease of use).
        ciph_loi = [int(ciph[i:i+2], 16) for i in range(0, len(ciph), 2)]
        
        # find the most likely key length.
        key_len = keyLength(ciph_loi)        
        key = findKey(key_len, ciph_loi)

        # Decrypt the ciphertext and print.
        decrypt(key, ciph_loi)


if __name__ == "__main__":
    main()

ucrure =hat&allo>s ir to  ntetceptialmist e?eryrhingg Wirh th s cgpabi%ity* theivasr maj&rit of !umah com$unieatio's ate au=omarical%y ihgest,d wothou= tatgeti'g. Of I >antcd toisee&youriemaols o; yosr wi/e's&phon,, ajl I !ave&to d& is&use  ntetcept:. I&can .et our ,maijs, p(sswirds,iphohe re*ordu, cr,dit&card:. I&don'= waht toilivc in ( soeietyithar doe: thcse s&rt if th ngs(.. Iido hot w(nt ro li?e ih a w&rld&wher, evcryth ng O do (nd uay i: reeorde-. Tnat i: nor som,thihg I (m wollin. to&supp&rt ir li?e uhder.
